# YouTube Naija Satire Generator

This project scrapes the latest videos from a YouTube channel and generates satirical 'Naija' takes for each video using a local Ollama LLM or by using Openrouter models. The workflow is designed for local development and testing, with modular sections for scraping, prompt engineering, and LLM integration.

---

## Project Outline

1. **Introduction & Motivation**
   This notebook aims to blend the power of LLMs with Nigerian pop culture by generating witty, satirical takes on trending YouTube videos. The goal is to showcase how AI can be used for creative, culturally relevant content generation, and to provide a fun, hands-on example of prompt engineering and LLM integration for local development.

2. **Environment Setup**
   To run this notebook, ensure you have Python 3.8+ installed. Install dependencies with `uv pip install -r requirements.txt` or directly in the notebook. You also need Ollama running locally (see https://ollama.com/ for installation instructions). Make sure to set up your .env file with any required API keys if you want to use OpenRouter or other APIs.

3. **YouTube Channel Scraping**
   The notebook uses the YouTube RSS feed to fetch the latest videos from a given channel ID. It extracts metadata such as title, description, and video URL for the most recent N videos. This approach avoids the need for YouTube API keys and is suitable for public channels.

4. **Prompt Engineering for Satire**
   A custom prompt template is crafted to instruct the LLM to generate satirical, Naija-style commentary. The prompt encourages the use of Pidgin English, local references, and humor, while maintaining a respectful and analytical tone.

5. **Local Ollama LLM Integration**
   The notebook connects to a local Ollama server using the OpenAI-compatible API. Prompts are sent to the LLM, and responses are received in real time. This setup ensures privacy, speed, and flexibility to swap models as desired.

6. **Satirical Take Generation**
   For each video fetched, the notebook generates a satirical take using the LLM and displays the result alongside the video title and link. This demonstrates how LLMs can be used for batch content generation and creative applications.

7. **User Controls & Output Formatting**
   Users can adjust parameters such as the number of videos to process, the channel ID, and the satire style by modifying variables in the notebook. Results are displayed in markdown for readability, with links to the original videos.

8. **Testing & Evaluation**
   The notebook can be run end-to-end with a sample channel to evaluate the quality and humor of the generated satire. Users are encouraged to experiment with different channels, prompt tweaks, and models to refine the output.

---

## Notes
- All LLM calls are routed to a local Ollama instance for privacy and speed.
- The notebook is modular: you can swap out the LLM or scraping logic as needed.
- Add your code in the indicated sections to build out each part of the workflow.
- For best results, use a modern LLM with good instruction-following and creative capabilities.
- This project is for educational and entertainment purposes only; always respect content creators and avoid generating harmful or offensive outputs.


In [27]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown


In [28]:
load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")
client = OpenAI(
  base_url = "https://openrouter.ai/api/v1",
  api_key = api_key
)

In [29]:
import yt_dlp

def get_channel_id(handle):
    url = f"https://www.youtube.com/{handle}"
    with yt_dlp.YoutubeDL({"quiet": True}) as ydl:
        info = ydl.extract_info(url, download=False, process=False)
        return info.get("channel_id") or info.get("uploader_id")



In [38]:
def get_channel_ids(handles):
    channel_ids = []
    for handle in handles:
        channel_id = get_channel_id(handle) 
        channel_ids.append(channel_id)
    return channel_ids

In [37]:
HANDLES = ["@AriseNewsChannel", "@NewsCentralAfrica"]  # YouTube handles to scrape
MAX_VIDEOS_PER_CHANNEL = 5  # max videos to fetch per channel

In [39]:
import feedparser

def get_latest_videos(channel_id, max_results=5):
    feed = feedparser.parse(f"https://www.youtube.com/feeds/videos.xml?channel_id={channel_id}")
    videos = [
        { "title": entry.title,
          "summary": entry.summary,
          "link": entry.link
        }
        for entry in feed.entries[:max_results]
    ]
    return videos


channel_ids = get_channel_ids(HANDLES)
latest_videos = []
for cid in channel_ids:
    for v in get_latest_videos(cid, max_results=MAX_VIDEOS_PER_CHANNEL):
        latest_videos.append({**v, "channel_id": cid})

In [40]:
system_prompt = """
You are a sharp, witty Nigerian satirist — think Falz meets Naija Twitter.
When given a YouTube video title and description, you write a short satirical take on it.
Use Pidgin English, local references, and current Naija slang where it fits naturally.
Respond in markdown. Do not wrap in a code block.
"""

In [41]:
def naija_take(video, model_name="openai/gpt-5-nano"):
    user_prompt = f"""
    Here is a YouTube video title and description. 
    Title: {video['title']}
    Description: {video['summary']}
Give a naija style satirical comic take on the video, be analytical and witty at the same time passing your information without hate.
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    response = client.chat.completions.create(
        model = model_name,
        messages=messages
    )
    result = response.choices[0].message.content
    display(Markdown(f"### [{video['title']}]({video['link']})\n{result}"))

In [42]:
# Example: Test with a different model (e.g., "llama2")
for video in latest_videos:
    naija_take(video)

### [Sports Segment: Barca Go Top Again As Madrid Lose and Atletico Wins](https://www.youtube.com/watch?v=_jPNI1B550A)
- Barca go top again? Oya, na so football waka: dem climb the ladder like person wey just find second-hand ladder for market. Madrid slump like phone battery after heavy Instagram scroll, while Atletico dey flex like dem don finally find the correct key for locked gate.

- E no be magic o; na points matter for this league, and Barca dey collect am small-small like market women wey sabi the correct price for everything. Madrid dey look for charger for their form, but the battery life no dey.

- Atletico win show say dem still dey grind wella; dem just dey keep the spare tire for the boot, ready to roll when another drama drop. Simeone dey run the show like coach wey sabi where to plant the knee and kick the drama away.

- If una wan gist fully, follow ARISE TV: subscribe for high profile interviews and all di social media plugs. Abi you think dvds dey grow wings? No be everything you need to know, na the gist from ARISE.

- Final take: the La Liga table dey dance like Naija wedding reception—everybody dey swing, nobody sure where the next move go land. Watch the drama, grab the tea, and no forget to press subscribe so you no miss when the next twist drop.

### [National Attention Increased Voter Participation in FCT Election - Sambo](https://www.youtube.com/watch?v=hcbjwbluRhQ)
Wahala for Abuja: Sambo don waka come with “National Attention Increased Voter Participation in FCT Election” as if attention na the new ballot paper. Na so this one dey? If na attention, then e go be like fine gel for camera: the glow dey, the message soft, but the real question still dey: how many people go actually carry PVC come vote?

Analytical gist time:
- Correlation vs causation. E fit be say media noise dey hype turnout, but we still need real factors: PVC collection, polling-unit accessibility, weather, and if the candidate epp anything wey people interpret as “change.” The video no prove this trend; e just show say “attention” dey, and somehow turnout dey there too. Abi na coincidence?
- Marketing meets democracy. The video na marketing strategy dressed as civic education: subscribe to ARISE TV for high-profile interviews, then maybe vote. If the attention push people to go out and vote, fine; if not, e simply na vibes. Either way, the numbers no lie on their own, we need the data.
- Sambo as brand. If Sambo be the magnet for the crowd, una no go lie: in Naija, people vote for the vibe as much as for the policy. Moral: policy matters, but the branding and media coverage na the booster rocket.

Punchline: Dem dey push attention as if na the magic wand to democracy. E no bad to get media buzz, but make we still verify the numbers, collect PVCs, and show up at the polling units. If you want more “high profile interviews,” sure, ARISE TV dey there. If you want the real thing, go vote, not just watch. We go dey look how e go play out—naija style: with plenty talk, slight shade, and hope say turnout serious reach.

### [FCT Election: Low Voter Turnout Worrisome - Nwangwu](https://www.youtube.com/watch?v=Zub7aBS7aTE)
- E reach: FCT elections waka smooth like Sunday service, but turnout dey low pass diet. We con dey wonder whether Abuja still get fans or dem just dey try collect new tech brooch for democracy. 
- For di six area councils (Abaji, Bwari, Gwagwalada, Kuje, Kwali, and AMAC), na cosmopolitan area wey dey hustle day and night—people dey work for city but dey live for suburb—dem still find time to skip voting. Democracy don jam brain with commute, no be small thing. 
- Curfew be the real gatekeeper. E lock many people out like door wey dem forget to unlock for party. People no fit waka to polling unit because night shift still dey run; e be like say curfew get PhD in voter suppression. 
- Observers talk say elections peaceful; no bad juju, just quiet. The only drama na the turnout. Na so e be if your party never invite you to the dance floor, you self dey stand gidigba for corner and dey watch. 
- Ezenwa Nwangwu, wey dey run Peering Advocacy and Advancement Centre in Africa, talk say civic responsibility na real wahala: make people vote; not voting na actually voting for something else. Na so dem dey push am: no vote, na you vote for the status quo, you just sip belle with your eyes closed. 
- Take-home: if you want change, you must show; if you no show, you no get right to grumble about who govern. Democracy no be Netflix—no skip episodes, you no fit rewind. 
- Final gist: the video dey remind us say peaceful day no mean we don finish; e just mean say una suppose show your own voice with ballot paper. Make we dey ready for next round, and yes, dem go still dey talk about it on ARISE TV.

### [Business Segment: Nigeria MPC Decision Due Tuesday](https://www.youtube.com/watch?v=uoBEZV_ZhW0)
Tuesday go show us the MPC drama again, as Nigeria’s money people decide the fate of our pockets: hold, cut, or hike the policy rate. Naija version of big boy politics but with more spreadsheets and less drama—yet the drama still plenty.

- The quick gist: Monetary Policy Committee dey look at inflation, growth, and exchange rate pass paraffin. If dem hold, money cost remain high; borrowing go still dey expensive and inflation fit slow but stubborn. If dem cut, money go cheaper, people fit borrow to buy things and expand business; but inflation fit remix like remix wey dey never finish. If dem hike, loan rates go up, growth go slow, and inflation might get some chill, but your wallet go shout.

- How e affect you: if you borrow for business or house, today decide tomorrow pay. If you save, you dey wait for rates to move; if you dey cash-and-carry, you dey watch naira and prices like hawk. Inflation na the stubborn baboon wey no wan sit down; policy decisions just try to tame am, not finish am.

- The real talk wey your auntie for market go whisper: anything can happen Tuesday, and markets go react faster than Lagos traffic. The outcome matter, but even the best plan still need cushion for surprises.

For more heat and real gist, the channel still dey: subscribe for high-profile interviews, and follow ARISE TV on Twitter, Instagram, and Facebook. And make you check www.arise.tv too—because Naija money talk no dey finish, e just change the channel.

### [The Morning Show: APC Wins 5, PDP Takes One in FCT Elections](https://www.youtube.com/watch?v=jS97JUEanyI)
Naija morning show don drop again: APC win 5, PDP take one for FCT. E be like class teacher shouting “five for the group project” while the class captain still dey argue who carry the marker. Abuja to the world, politics still be the original reality show.

- The math no balance: 5-1 for FCT be like “waka pass” math wey even Google know say na arithmetic wey dey play hide-and-seek. For capital city, dem fit dono give APC five seats and PDP one, but question still dey: who carry the mic for the next episode? 
- Abuja matter: FCT no be state, e be capital. Elections for the center no easy—na who dey run the center? Everybody go still dey wait for the next drumroll, or maybe the next press conference where someone go describe “algorithms” wey count the ballot papers with the precision of someone counting kobo coins.
- Media theatre: The Morning Show no go shy to carry high profile interviews, even as the result waka enter. Na so dem fit say: “subscribe to our channel for more interviews,” while the real drama na the tally wey dey unfold. If you wan the long-waited drama, follow ARISEtv on Twitter, Instagram, Facebook, and their website—basically, na democracy livestream with social media interludes.

Moral of the story: Elections dey, results dey, drama dey—na standard Nigerian season. If you want more gist, you know the channels to click; but make you no forget to laugh small and still carry your PVC like passport to the next round.

### [#FCTDecides2026: APC wins 5 councils, PDP takes Gwagwalada](https://www.youtube.com/shorts/zCEM-BLX_0g)
Ah! Abuja politics still dey remix like old house of ills. #FCTDecides2026: APC win 5 councils, PDP take Gwagwalada. Na so the map burst, like popcorn for cinema.

- APC still sabi ground game pass; five councils na real estate, dem fit show say dem still get the machinery to move vote, petrol price no finish to kobo the work.
- PDP grabbing Gwagwalada show say urban/central areas still dey swing and young people fit turn up when dem go bring jollof and loud music for campaign; Abuja no be one-dish kitchen.
- E sweet to see the contrast: rural-leaning blocks vs urban-core space; na politics of the ground, not just poster for social media. No be hype alone, na local issues dey drive.
- The bigger takeaway: Abuja politics still top level of “who dey ride the wheel” but the wheel dey wobble for the city; swing seats dey, and no one fit shout turn by turn.
- And as the counting drama dey continue, make we keep popcorn ready: the real story no finish until all results land, and the next campaign slogan go ready to drop.

Bottom line: power distribution still get spice, and Gwagwalada show say even for FCT, e be aje butter on bread—different tastes for different corners.

#ElectionSatire #NaijaPolitics #Abujaupdate

### [Somaliland is ready to grant US access to minerals and bases](https://www.youtube.com/shorts/JGX_s8s5wGM)
Gist for you: Somaliland don yan the US compass, say make dem waka into minerals and bases - na real or na headline breeze? Here my Naija take, sharp, analytical and with small spice.

- Minerals go turn to: US dey look for lithium, rare earths, gold, you name am. E no be only “minerals”, e be “power to run the world’s gadgets hole” vibes. If dem sign, na the global supply chain go shake pass as If dem just leak the battery for your phone and say “you’re welcome.” The question wey dey front: who go dey benefit for real, and how dem go protect di land from turning to big mining site with road signs saying “I go collect this brain for export”?

- Bases matter: Bases no be for village people to hold tengue meetups. E mean logistics hubs, surveillance, and drone corridors. If US fit base Somaliland, na geopolitical chess we dey play: security umbrella for some regions, but also potential flashpoint for others wey dey look am like “why you dey park your tanks here?” Local communities go gaslight themselves as “development” or “resource extraction with a side of noise.”

- Sovereignty and recognition: Somaliland no be fully recognised by many nations, so any deal with the US go require careful diplomatic wiring. E go resemble “we dey talk to Uncle Sam; but our house still under reconstruction.” For global politics, recognition levels fit determine how sweet or sour the arrangement go taste. Naija own sabi: wahala if agreement no clear-cut, we finish with more questions than answers.

- Local impact: If minerals and bases come, e go bring roads, schools, or e go bring enviromental gbege wey turn nearby communities into the first test sites for “how to coexist with heavy machinery.” Naija fuller ground: the sweet talk of development must come with accountability, fair benefit sharing, and real community consent. No be only press release matter.

- Nigeria angle wey dey pata: If this model work, people go dey ask, “how we fit replicate for our own resources?” But also dem go fear “debt-for-access” arrangements and sovereignty questions. The real win go be if resource wealth translate to real local development, not just another foreign-funded project wey vanish before elections.

- Final gist: The video dey spicy and dramatic, but geopolitics na a big game with many players and few straight answers. If Somaliland and US pull this off, expect a track record of deals where the headlines look glamorous and the details require reading glasses. For now, e remain geopolitics dressed as mineral shopping – naija watch-and-see, with popcorn ready and sense on standby.

### [Somaliland minister says US could gain access to minerals and military sites](https://www.youtube.com/shorts/dtvGQK1WcLQ)
Somaliland minister say US fit gain access to minerals and military sites. Na so market talk begin for news wey dey sound like “make una fetch oil for yesterday.” Abeg, na who take the key? What kind of access be that—like you go just waka up include dem for your passport?

- In plain Naija: dem dey try sell us “go share market” with foreign power, but sovereignty still dey, like Lagos traffic—everybody dey claim roads, but dem need permit.
- Minerals dey? E mean say mineral wealth fit become American loan shark—dem go come collect fees in dollars while we dey beg for petrol money.
- Military sites for road? That one na base access, which require serious agreements, no be gist from press conference and chant. E resemble “SOFA” matter you for sign before you dey allowed near generator.
- The reality check: this kind talk dey show how global power plays with small states—everybody dey hustle for strategic sitting room, while the local people still dey look the TV without charging their phones.
- Naija lesson: if your country no sign proper agreements and local employment provisions, you go just dey watch as base near your backyard, and you go still dey ask how to fix potholes for your own street.
- Final gist: make we dey sharp; sovereignty no be giveaway. If US or any country wan gain access, dem go show contracts, transparent benefit, and real local benefits; otherwise e go resemble “open sesame” wey nobody check who dey behind the door.

Bottom line: World waka, but make we no forget to protect our own garden before we dey open gate for visitors.

### [Candidates and voters react after FCT Council results are released](https://www.youtube.com/shorts/SMaQyNks1eo)
Abuja drama don enter second season: FCT Council results don drop, and the reactions na full sitcom. Here be the Naija-style take as e dey unfold.

- Winners dey gather for press corner like dem don find Wi-Fi signal for shrine, smiling with their promises wey sound like loudspeaker for market square: road blocks removed, schools upgraded, and a mini power plant in every ward—abi na pencil sketch? Time go tell which one go reach the light.

- The winner’s speech dey try hold serious face, but the pockets of joy show for eye. Dem talk about “tactical responsiveness” and “youth empowerment,” while the only thing wey most people want na stable light, safe streets, and no go slow ambulances.

- Runner-up waka go court of public opinion, but na social media be their courtroom. Memes anchor the narrative as if comedy central. The caption: “Abuja, make I borrow your broom before you sweep me join the dustbin.” The vibe: win or lose, the headline still be drama.

- Ward-level kandidats dey hustle too: some go act like dem don win, others go dey blame “tactical miscommunication” and vow to "re-strategize." Na so politics dey: plenty talk, little action, more tempo for the next promo video.

- Voters side: queues long like morning for Bank, but some people just dey there with optimism like dem drink tea and forget pepper. Some collect small sweet talk and a bottle of water; others just dey recall the cost of living while casting ballot.

- The social media chorus: “This is change!” followed by, “Make the change reach us o.” People dey pair promises with receipts—budgets, development plans, and ward-level projects—yet the cynicism still dey pepper the conversation.

- Demography angle: youth dem show interest, old folks still hold the fort. The question wey everybody dey ask: who go really monitor the money, who go keep policy steady, and how this one go affect Abuja wey never finish glossing over potholes.

- Governance reality check: campaign promises fit resemble sugar for the mouth, but governance na the real pepper. If this one go deliver tangible road repair, better healthcare, and safe neighborhoods, e go require steady oversight beyond the next election promo video.

- Takeaway: elections na theatre, but accountability na the true script. The vote na opening act; the real show be how council leaders deliver. So make we dey ready to follow budgets, track project progress, and hold dem to the promise wey fit improve our streets, market, and schools.

Naija politics, still spicy. We go see how e go unfold—no hate, just the waka wey dey show us say, e dey possible to laugh while we dey push for real change.

### [FCT Poll Aftermath: ‘Mutilated’ Results Found on INEC's Portal](https://www.youtube.com/watch?v=_fTzNr5MNQw)
Wahala dey, but make we joke small. Here be Naija-style comic take on the matter:

So, FCT poll waka come, and dem done find mutilated results for INEC portal. Na drama wey fit pass telenovela: the system wey supposed to print the truth dey hide like person wey borrow your charger and never return am. On top this, the Wake Up Conversations crew—Gbenga, Olive, and Joe—don talk say tech fit help us know who win, but e no dey solve trust if our oversight no strong. BVAS dey shine like new iron, yet the online results look like them don use eraser βeta to tidy the numbers. Kai.

Now some quick observations, in plain Naija:

- The paradox: digital portals supposed to be the "no-foolery" gate, but human beings still dey behind the keyboard. If you fit print something, you fit alter am. E no be only about “we have better gadgets” o; e be about who dey watch the gadget. If oversight no tight, even the best interface go still look like lie detector wey dey beep at the wrong time.

- Public confidence vs. tech glow: People go dey ask, “If the portal show mutilated results today, which party go trust the system tomorrow?” Dem talk about transparency, audit trails, and independent checks. Dem want something like: read-only access for observers, tamper-evident logs, end-to-end verifiability. Not just “BVAS works well” with a side dish of suspicious screenshots.

- The politics of debate: Parties and observer groups dey shout; dem want thorough investigation. Na so independence suppose be. If we go give system all the credit and refuse to investigate when something fishy appear, e go resemble person wey praise his kulikuli just because e smell nice—while the fish dey for another plate.

- Practical fixes wey make sense: 
  - Real-time, verifiable audit trails wey investigators fit examine without begging for passcodes. 
  - Independent cyber-oversight plus post-election forensic reviews. 
  - Clear consequences for tampering; e good to show say democracy na serious business, not Lagos street market where anything dey happen.

- The bigger picture: Technology can boost efficiency, but credibility still hinge on accountability. Portals fit do fine, but if people no dey answer when wrong things show, trust go dey on a long vacation. Na hand we go use to steady the ship, not just gadget wey dey shine for the sun.

Wetin this mean for future elections? Keep the shine of BVAS, but add the spine of oversight. Make we improve data governance, ensure independent verification, and make it easy for observers to verify counts without turning states into a guessing game. Abeg, let us move from “the numbers look good on screen” to “the numbers reflect the will of the people, and there are verifiable receipts if you ask politely.”

Final words, make we remember: the gospel of democracy no be “nice interface”; e be accountability, traceable records, and real consequences for anyone who tampers. Until then, we go dey watch, dey gbo, and dey hope say the next time the portal no go resemble someone edit the results with a red pen during siesta.

News-wise, una fit keep eye on the calls for thorough investigation and stronger oversight—because na so we go reach the point where trust just dey grow, not just the download speed of the portal. And if all else fail, at least we still get memes.